In [111]:
from datetime import datetime
import pandas as pd
from geopy import distance
import numpy as np

import random

import pymongo
from pymongo import MongoClient

In [112]:
client = MongoClient('localhost', 27017)

db = client['avl_pipeline_test']
start_coll = db['redux_starts']
trip_coll = db['redux_labeled_trips']

In [113]:
all_trip_ids = trip_coll.distinct('trip_id_iso')

In [114]:
rand_200 = random.sample(all_trip_ids, 200)

In [130]:
chunk_ends = []

for samp_id in rand_200:
    trip_start = start_coll.find_one({"trip_id_iso": samp_id})
    start_ts = trip_start['time_stamp']
    frst_tm_chnk = start_ts + 245
    
    search = {
        'trip_id_iso': samp_id,
        'time_stamp': {"$gt": frst_tm_chnk}
    }
    
    chnk_end = list(trip_coll.find(search).sort("time_stamp"))
    
    if chnk_end:
    
        chunk_ends.append((chnk_end[0]['LATITUDE'], chnk_end[0]['LONGITUDE']))  

In [131]:
chunk_ends

[('37.7821', '-122.45896'),
 ('37.78391', '-122.45912'),
 ('37.78691', '-122.4565'),
 ('37.78419', '-122.45913'),
 ('37.7824', '-122.45901'),
 ('37.77443', '-122.45731'),
 ('37.783', '-122.45909'),
 ('37.78519', '-122.45919'),
 ('37.78226', '-122.45897'),
 ('37.785', '-122.45926'),
 ('37.78282', '-122.45903'),
 ('37.7825', '-122.459'),
 ('37.7838', '-122.45912'),
 ('37.78301', '-122.45911'),
 ('37.78397', '-122.45911'),
 ('37.78283', '-122.45907'),
 ('37.78144', '-122.45896'),
 ('37.78294', '-122.45905'),
 ('37.78573', '-122.45892'),
 ('37.78295', '-122.45913'),
 ('37.78315', '-122.45908'),
 ('37.7857', '-122.45915'),
 ('37.78576', '-122.45906'),
 ('37.78358', '-122.45907'),
 ('37.78571', '-122.45894'),
 ('37.78696', '-122.45662'),
 ('37.78583', '-122.45871'),
 ('37.78213', '-122.45899'),
 ('37.78037', '-122.45881'),
 ('37.78185', '-122.45898'),
 ('37.78478', '-122.45916'),
 ('37.784', '-122.45913'),
 ('37.78138', '-122.459'),
 ('37.78398', '-122.45915'),
 ('37.77445', '-122.4584'),
 (

In [135]:
seq = pd.read_csv('stop_sequence.csv')
stops = pd.read_csv('data/gtfs/sfmta_2017-02-10/stops.txt')

test_section = seq.iloc[1:5]

stop_testing = {}

for row in seq.iterrows():
    
    stop_distances = []
    
    stop_id = row[1]['stop_id']
    stp_info = stops[stops['stop_id'] == stop_id]
    
    lat = stp_info['stop_lat'].values
    lon = stp_info['stop_lon'].values
    
    for location in chunk_ends:
        
        stop_distances.append(distance.distance((lat, lon), location).m)
        
    numps = np.array(stop_distances)
    
    print ("Stop: ", stop_id)
    print ("Average Distance: ", numps.mean())
    

Stop:  6293
Average Distance:  415.8094734604828
Stop:  3879
Average Distance:  453.1704814625828
Stop:  3852
Average Distance:  338.3373035266208
Stop:  3644
Average Distance:  229.59840378806908
Stop:  3645
Average Distance:  210.11579323084226
Stop:  3649
Average Distance:  333.9400676067834
Stop:  3642
Average Distance:  783.3059254797572
Stop:  4224
Average Distance:  1082.4623947808068
Stop:  6479
Average Distance:  1111.481981964013
Stop:  6481
Average Distance:  1310.6223297567558
Stop:  4963
Average Distance:  1718.819772803967
Stop:  4947
Average Distance:  1820.4704133576693
Stop:  3665
Average Distance:  1954.8308825450692
Stop:  7295
Average Distance:  2144.4735898012545
Stop:  3663
Average Distance:  2352.5870296715166
Stop:  3657
Average Distance:  2448.2737270397265
Stop:  7220
Average Distance:  2541.5933448540077
Stop:  4070
Average Distance:  2774.5230673101087
Stop:  4076
Average Distance:  3017.877533565865
Stop:  4080
Average Distance:  3116.023222870721
Stop:  56

In [136]:
stops[stops['stop_id'] == 3645]

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url
707,3645,Arguello Blvd & Clement St,,37.783071,-122.45907,,


In [102]:
trip_coll.find({'trip_end':1}).count()

1332